# Introduction
#### she changlue
20th April 2017

This project is construct to handle documents topic classification.
I use self designed algorithm to efficiently embedding the documents' topic vector and tokens' topic vector.It is emperiacally proved that this algorithm can classify the topic of different documents and tokens in a speed way.

this notebook will process as follow:
1. load library and raw corpus data
2. cut the corpus in to a list format
3. encode the tokens and corpus
4. construct model and train
5. use kmeans to do tokens' and docs' cluster 
6. use T-SNE to visualization
7. save the outcomes

### 1)   load library and raw corpus data

In [21]:
% matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib

from sklearn.cluster import KMeans#cluster

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import jieba.posseg as pseg # cut the documents with token and tags
import jieba
import tensorflow as tf

from sklearn.manifold import TSNE#cluster

from subprocess import check_output
print(check_output(["ls", "corpus"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

custormTokens.txt
opr_rem.csv
催收sample.csv
马上消费金融_jiuhui.wu_2017-04-05.csv



In [22]:
jieba.load_userdict('corpus/custormTokens.txt')  
rawdata = pd.read_csv('corpus/opr_rem.csv', header=0)

#### corpus briefing

In [23]:
rawdata.head()

,opr_rem
0,今天下午5点472
1,\t告知客户电话15226088432
2,说是叔侄关系 转告
3,客户承诺今天下午五点存入184
4,已提醒征信影响。已告知从今天凌晨开始又会多增加75元的滞纳金。客户敷衍明天去解决


In [24]:
rawdata.shape

(999988, 1)

In [25]:
rawdata.describe()

,opr_rem
count,999986
unique,999986
top,"卡里有钱 450元,划扣，剩余的321明天下午五点之前存入银行卡"
freq,1


### 2) cut the corpus in to a list format

In [26]:
tokenCorpus  = []#corpus list of cutted tokens
rawSentences = []#raw text 
#documents    = list(rawdata[rawdata['消息目标']=='机器人']['消息内容'])#text which is send by custormers
documents    = list(rawdata['opr_rem'])#text which is send by custormers

In [ ]:
# cstruct the corpus
for sentence in documents:
    if len(str(sentence))>4:
        sentence = sentence.replace('\t','')
        words = [pair.word for pair in pseg.lcut(sentence) if pair.flag in ['n','ns','vs','nv','v']]       
        if len(words)>1:
            tokenCorpus.append(words)
            rawSentences.append(sentence)

In [ ]:
tokenCorpus[:3]

In [ ]:
rawSentences[:5]

### 3) encode the tokens and corpus

In [ ]:
HP_miniTokenFreq = 3 #minimal tokens frequency
tokenCount = dict()  #token count
token2code = dict()  #token to code
code2token = ['inFreqTokens'] #code to token
code       = 1       #code

##### transfer token into codes

In [ ]:
#get token frequency
for tokens in tokenCorpus:
    for token in tokens:
        tokenCount.setdefault(token,0)
        tokenCount[token]+=1  
#encode those tokens which have minumal frequency
for token in tokenCount:
    if tokenCount[token] > HP_miniTokenFreq:       
        token2code[token] = code
        code += 1
        code2token.append(token)
    else:
        token2code[token] = 0
#transfer the raw token corpus into encoded corpus
codeCorpus = []
for tokens in tokenCorpus:          
    codeCorpus.append([token2code[token]for token in tokens])

### 4) construct model and train

In [ ]:
HP_topicNums = 8
documNums = len(codeCorpus)
tokenNums = len(code2token)
print("the training documents num:",documNums)
print("the training tokens    num:",tokenNums)

##### dense list to sparse numpy

In [ ]:
tokenDocMat = np.zeros(shape=(documNums,tokenNums),dtype=np.float32)
for docIdx in range(documNums):
    tokenDocMat[docIdx][codeCorpus[docIdx]]=1
docTokenNum = tokenDocMat.sum(axis=1).reshape((-1,1))   

In [ ]:
docIndxSpan = np.array(range(documNums),dtype=np.int32)

##### construct the graph

In [ ]:
#embedding initial
docum2topicEmbed  = tf.Variable(tf.random_uniform([documNums,HP_topicNums], -1.0, 1.0))
token2topicWeight = tf.Variable(tf.random_uniform([HP_topicNums,tokenNums], -1.0, 1.0))

In [ ]:
#placeholder
document_idx   = tf.placeholder(tf.int32, shape=[None])
document_embed = tf.nn.embedding_lookup(docum2topicEmbed , document_idx) 
multiplier     = tf.placeholder(tf.float32, shape=[None,1])
tokensInDoc    = tf.placeholder(tf.float32,shape=[None,tokenNums])

In [ ]:
#forward 
with tf.name_scope("TM"):
    documentembed_Sfmx     = tf.nn.softmax(document_embed)
    token2topicWeight_Sfmx = tf.nn.softmax(token2topicWeight,dim=0)
    tokensInDoc_pred       = tf.nn.softmax(tf.matmul(documentembed_Sfmx,token2topicWeight_Sfmx))*multiplier                           

In [ ]:
# use least squre loss to train model
with tf.name_scope("loss"):   
    loss = tf.reduce_mean(tf.square(tokensInDoc - tokensInDoc_pred))

In [ ]:
learning_rate = 0.001
with tf.name_scope("train"):
    optimizer   = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [ ]:
#initial and save
init = tf.global_variables_initializer()
saver = tf.train.Saver()

#### training the tensorflow graph

In [ ]:
#start training
n_epochs   = 100
batch_size = 1000
batch_num  = documNums//batch_size+1
 
with tf.Session() as sess:    
    init.run()
    for epoch in range(n_epochs):
        for batch in range(batch_num):
            #feed rawdata
            document_idx_batch = docIndxSpan [batch*batch_size:(batch+1)*batch_size]
            multiplier_batch   = docTokenNum [batch*batch_size:(batch+1)*batch_size]
            tokensInDoc_batch  = tokenDocMat [batch*batch_size:(batch+1)*batch_size]
             
            #training
            sess.run(training_op, feed_dict={document_idx : document_idx_batch[:20000],
                                             multiplier   : multiplier_batch[:20000],
                                             tokensInDoc  : tokensInDoc_batch[:20000]})  
        if epoch%100==0:
            train_loss = loss.eval(feed_dict={document_idx : docIndxSpan,
                                              multiplier   : docTokenNum,
                                              tokensInDoc  : tokenDocMat})
            print(train_loss)
    save_path = saver.save(sess,"tfsave/topic_model.ckpt")
    # get embedding
    documMat = documentembed_Sfmx.eval(feed_dict={document_idx : docIndxSpan})
    tokenMat = token2topicWeight_Sfmx.eval()

In [ ]:
documMat.sum(axis=1)[:5],tokenMat.sum(axis=0)[:5]

#### show the rough topic

In [ ]:
topK = 3
outDF = []
keyWords = []
keySentences = []
for idx in range(HP_topicNums):    
    ords = np.argsort(-tokenMat[idx])[:topK]
    tmp = ['主题'+str(idx+1)]
    tmp += [code2token[i]for i in ords]
    keyWords+=tmp
    ords = np.argsort(-documMat.T[idx])[:topK]
    tmp = ['------------------------------']
    tmp +=[rawSentences[i]for i in ords]
    keySentences+=tmp
outDF = [keyWords,keySentences] 

In [ ]:
pd.DataFrame(outDF).T.head(30)

### 5)  use kmeans to do tokens' and docs' cluster

In [ ]:
tokenMat_KM = KMeans(n_clusters=HP_topicNums, random_state=0).fit(tokenMat.T)
documMat_KM = KMeans(n_clusters=HP_topicNums, random_state=0).fit(documMat)

In [ ]:
tokenMat_Label = tokenMat_KM.labels_
documMat_Label = documMat_KM.labels_

### 6)  use T-SNE to visualization

In [ ]:
tokenMat_tsne_embed = TSNE(random_state=1).fit_transform(tokenMat.T)  
documMat_tsne_embed = TSNE(random_state=1).fit_transform(documMat)  

##### the visualization of word embeddings

In [ ]:
plt.scatter(tokenMat_tsne_embed[:,0],tokenMat_tsne_embed[:,1],c=tokenMat_Label, cmap=plt.cm.get_cmap("jet", 10))

##### the visualization of document embeddings

In [ ]:
plt.scatter(documMat_tsne_embed[:,0],documMat_tsne_embed[:,1],c=documMat_Label, cmap=plt.cm.get_cmap("jet", 10))

### 7)  save the outcomes

In [ ]:
outDF= pd.DataFrame()
outDF['text']=rawSentences
outDF['label']=documMat_Label

In [ ]:
outDF.head()

In [ ]:
outDF.to_csv('save/docLabel.csv',encoding='gbk',index=False)